In [123]:
import os
import cv2
import random
import rasterio
import numpy as np
from PIL import Image, ImageEnhance
import geopandas as gpd
from datetime import datetime

In [144]:
def getEnhancedDataPath(dataPath):
    '''  
    get all enhanced data path as list
    '''
    EnhancedDataPathList = []
    #loop into day folder
    for mainFolder in os.listdir(dataPath):
        mainFolderPath = dataPath + '/' + mainFolder

        #loop into label folder
        for labelFolder in os.listdir(mainFolderPath):
            labelFolderPath = mainFolderPath + '/' + labelFolder

            # loop into every file
            for dataFile in os.listdir(labelFolderPath):

                # check enhanced folder
                if(dataFile == 'Enhanced'):
                    enhancedFolderPath = labelFolderPath + "/" + dataFile

                    # loop into enhanced folder
                    for enhancedFile in os.listdir(enhancedFolderPath):
                        enhancedFilePath = enhancedFolderPath + "/" + enhancedFile
                        EnhancedDataPathList.append(enhancedFilePath)
                
        # print(mainFolder)
    return EnhancedDataPathList

def readJPGImg(path):
    '''
    read jpg and return image
    '''
    try:
        # check if image exist, then read and return image
        image = cv2.imread(path)
        return image
    except:
        print("could not read image")

def saveJPGImg(path, image):
    '''
    save jpg image
    '''
    # check if image already exist, if not, save image
    if(not(os.path.exists(path))):
        cv2.imwrite(path, image)
        print(f"image saved: {path}")
    else:
        # image already exist
        print(f"image already exist: {path}")

def rotate_image(image, angle=180):
    """
    Rotate the image by the specified angle.

    Parameters:
    - image: Input image (NumPy array)
    - angle: Rotation angle (default 180°)

    Returns:
    - Rotated image
    """
    height, width = image.shape[:2]
    center = (width // 2, height // 2)  # Find the center of the image

    # Compute the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Perform the rotation
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height), borderMode=cv2.BORDER_REFLECT)

    return rotated_image

def flip_horizontal(image):
    """
    Applies horizontal flip to the given image.
    
    Parameters:
    - image: Input image (NumPy array)

    Returns:
    - Flipped image
    """
    return cv2.flip(image, 1)  # Flip along vertical axis (left ↔ right)

def apply_zoom(image, zoom_factor=1.2):
    """
    Applies zoom to the given image.

    Parameters:
    - image: Input image (NumPy array)
    - zoom_factor: Scale factor (>1 for zoom-in, <1 for zoom-out)

    Returns:
    - Zoomed image (same size as input)
    """
    height, width = image.shape[:2]

    # Compute new zoomed dimensions
    new_width = int(width / zoom_factor)
    new_height = int(height / zoom_factor)

    # Compute cropping coordinates
    x1 = (width - new_width) // 2
    y1 = (height - new_height) // 2
    x2 = x1 + new_width
    y2 = y1 + new_height

    # Crop and resize to original size
    cropped = image[y1:y2, x1:x2]
    zoomed_image = cv2.resize(cropped, (width, height), interpolation=cv2.INTER_LINEAR)

    return zoomed_image

def apply_gamma_correction(image, gamma):
    """
    Applies gamma correction to adjust image brightness.

    Parameters:
    - image: Input image (NumPy array)
    - gamma: Gamma value (>1 for brighter, <1 for darker)

    Returns:
    - Adjusted image
    """
    inv_gamma = 1.0 / gamma  # Inverse gamma for LUT mapping
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(0, 256)]).astype(np.uint8)

    return cv2.LUT(image, table)  # Apply the gamma correction

def apply_color_jitter(image, brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1):
    """
    Applies Color Jitter (random changes in brightness, contrast, saturation, and hue).

    Parameters:
    - image: Input image (NumPy array, BGR format).
    - brightness: Max brightness change factor (e.g., 0.2 = ±20% change).
    - contrast: Max contrast change factor (e.g., 0.2 = ±20% change).
    - saturation: Max saturation change factor (e.g., 0.2 = ±20% change).
    - hue: Max hue shift factor (e.g., 0.1 = ±10% change).

    Returns:
    - Jittered image
    """
    # Convert BGR to HSV for color adjustments
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)

    # Adjust brightness (Value channel)
    brightness_factor = np.random.uniform(1 - brightness, 1 + brightness)
    hsv[:, :, 2] = np.clip(hsv[:, :, 2] * brightness_factor, 0, 255)

    # Adjust saturation (Saturation channel)
    saturation_factor = np.random.uniform(1 - saturation, 1 + saturation)
    hsv[:, :, 1] = np.clip(hsv[:, :, 1] * saturation_factor, 0, 255)

    # Convert back to BGR
    jittered_image = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)

    # Adjust contrast
    contrast_factor = np.random.uniform(1 - contrast, 1 + contrast)
    jittered_image = np.clip((jittered_image - 127.5) * contrast_factor + 127.5, 0, 255).astype(np.uint8)

    return jittered_image

def apply_gaussian_noise(image, mean=0, std=0.6):
    """
    Applies Gaussian noise to an image.

    Parameters:
    - image: Input image (NumPy array).
    - mean: Mean of Gaussian noise (default = 0).
    - std: Standard deviation of Gaussian noise (default = 15).

    Returns:
    - Noisy image
    """
    noise = np.random.normal(mean, std, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)  # Add noise to image
    return np.clip(noisy_image, 0, 255).astype(np.uint8)  # Keep values in valid range

def loopAugmentImage(targetPathList):
    ''' 
    call all the augmentation method for enhanced image and svae to "Augmented" folder
    '''
    count = 0

    # Get current time
    start_time = datetime.now()

    # loop into each target file path
    for targetPath in targetPathList:
        # create output path in Enhanced folder
        splittedPath = targetPath.split('/')
        prevFolderPath = splittedPath[:-2]
        filename = splittedPath[-1][:-4]
        # .../RGB/RGB_202401181250/RGB_202401181250_1/Augmented/RGB_202401181250_1_original_enhanced
        outFilePath = '/'.join(prevFolderPath) + "/" + "Augmented" + "/" + filename

        # read image
        targetImgOriginal = readJPGImg(targetPath)

        # 1 save original image as one of the data
        originalImageOutPath = outFilePath + "_original.jpg"
        saveJPGImg(originalImageOutPath, targetImgOriginal)
        
        # 2 rotate image for 180 degree and save in Augmented data
        rotatedImg = rotate_image(targetImgOriginal)
        rotatedImgOutPath = outFilePath + "_rotated.jpg"
        saveJPGImg(rotatedImgOutPath, rotatedImg)

        # 3 flip image horizontally and save image to Augmented data
        flippedImg = flip_horizontal(targetImgOriginal)
        flippedImgOutPath = outFilePath + "_flipped.jpg"
        saveJPGImg(flippedImgOutPath, flippedImg)

        # 4 zoom image for 20% and save to Augmented data
        zoomedImg = apply_zoom(targetImgOriginal)
        zoomedImgOutPath = outFilePath + "_zoomed.jpg"
        saveJPGImg(zoomedImgOutPath, zoomedImg)

        # 5 apply gamma correction 1.2 (bright) to original image and save to Augmented data
        brightenOriginal = apply_gamma_correction(targetImgOriginal, gamma=1.2)
        brightenOriginalOutPath = outFilePath + "_brightenOriginal.jpg"
        saveJPGImg(brightenOriginalOutPath, brightenOriginal)

        # 6 apply gamma correction 1.2 (bright) to flipped image and save to Augmented data
        brightenFlipped = apply_gamma_correction(flippedImg, gamma=1.2)
        brightenFlippedOutPath = outFilePath + "_brightenFlipped.jpg"
        saveJPGImg(brightenFlippedOutPath, brightenFlipped)

        # 7 apply gamma correction 0.8 (dark) to original image and save to Augmented data
        darkenOriginal = apply_gamma_correction(targetImgOriginal, gamma=0.8)
        darkenOriginalOutPath = outFilePath + "_darkenOriginal.jpg"
        saveJPGImg(darkenOriginalOutPath, darkenOriginal)

        # 8 apply gamma correction 0.8 (dark) to flipped image and save to Augmented data
        darkenFlipped = apply_gamma_correction(flippedImg, gamma=0.8)
        darkenFlippedOutPath = outFilePath + "_darkenFlipped.jpg"
        saveJPGImg(darkenFlippedOutPath, darkenFlipped)

        # 9 apply random changes in brightness, contrast, saturation, and hue and save to Augmented data
        jitteredImg = apply_color_jitter(targetImgOriginal)
        jitteredImgOutPath = outFilePath + "_jittered.jpg"
        saveJPGImg(jitteredImgOutPath, jitteredImg)

        # 10 apply gaussian noise and save to Augmented image
        noiseImg = apply_gaussian_noise(targetImgOriginal)
        noiseImgOutPath = outFilePath + "_noisy.jpg"
        saveJPGImg(noiseImgOutPath, noiseImg)


        count += 1
        current_time = datetime.now()
        time_elapsed = current_time - start_time
        time_left = (len(targetPathList)/count)*time_elapsed
        print(f"{count} / {len(targetPathList)} >> {count*100/len(targetPathList):.3f}% >> estimated time until finish: {time_left}")

In [146]:
RGBPath = "D:/ice-wheat/data/dataForProcess/mainData/RGB"
enhancedDataPathList = getEnhancedDataPath(RGBPath)
loopAugmentImage(enhancedDataPathList)

# enhancedImg = readJPGImg("C:/Users/pacha/Desktop/masterProj/MasterProj/augmentation/test3.jpg")

# rotateImg = rotate_image(enhancedImg)
# saveJPGImg("./rotate3.jpg",rotateImg)

# flipImg = flip_horizontal(enhancedImg)
# saveJPGImg("./flip3.jpg",flipImg)

# zoomImg = apply_zoom(enhancedImg)
# saveJPGImg("./zoom3.jpg",zoomImg)

# gammaImgOriBright = apply_gamma_correction(enhancedImg, gamma=1.2)
# saveJPGImg("./gamma3.jpg",gammaImgOriBright)

# gammaImgflipBright = apply_gamma_correction(flipImg, gamma=1.2)
# saveJPGImg("./gamma3.jpg",gammaImgflipBright)

# gammaImgOriDark = apply_gamma_correction(enhancedImg, gamma=0.8)
# saveJPGImg("./gammaD3.jpg",gammaImgOriDark)

# gammaImgflipDark = apply_gamma_correction(flipImg, gamma=0.8)
# saveJPGImg("./gammaD3.jpg",gammaImgflipDark)

# jitterImg = apply_color_jitter(enhancedImg)
# saveJPGImg("./jitter3.jpg", jitterImg)

# noiseImg = apply_gaussian_noise(enhancedImg,0,0.6)
# saveJPGImg("./noise3.jpg", noiseImg)


image already exist: D:/ice-wheat/data/dataForProcess/mainData/RGB/RGB_202401181250/RGB_202401181250_1/Augmented/RGB_202401181250_1_original_enhanced_original.jpg
image already exist: D:/ice-wheat/data/dataForProcess/mainData/RGB/RGB_202401181250/RGB_202401181250_1/Augmented/RGB_202401181250_1_original_enhanced_rotated.jpg
image already exist: D:/ice-wheat/data/dataForProcess/mainData/RGB/RGB_202401181250/RGB_202401181250_1/Augmented/RGB_202401181250_1_original_enhanced_flipped.jpg
image already exist: D:/ice-wheat/data/dataForProcess/mainData/RGB/RGB_202401181250/RGB_202401181250_1/Augmented/RGB_202401181250_1_original_enhanced_zoomed.jpg
image already exist: D:/ice-wheat/data/dataForProcess/mainData/RGB/RGB_202401181250/RGB_202401181250_1/Augmented/RGB_202401181250_1_original_enhanced_brightenOriginal.jpg
image already exist: D:/ice-wheat/data/dataForProcess/mainData/RGB/RGB_202401181250/RGB_202401181250_1/Augmented/RGB_202401181250_1_original_enhanced_brightenFlipped.jpg
image alrea

KeyboardInterrupt: 